In [96]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!', ',']
data_file = open('keywords.json').read()
keywords = json.loads(data_file)

In [97]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [98]:
for intent in keywords['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
    
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [99]:

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

35 documents
7 classes ['Deposit', 'Loan', 'credit card', 'goodbye', 'greeting', 'options', 'thanks']
57 unique lemmatized words ["'s", 'a', 'all', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'card', 'chatting', 'check', 'could', 'credit', 'day', 'deposit', 'do', 'find', 'for', 'get', 'gold', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'issue', 'later', 'list', 'loan', 'me', 'money', 'next', 'nice', 'offered', 'option', 'provide', 'see', 'show', 'sum', 'support', 'take', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'you']


In [100]:

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []

    pattern_words = doc[0]

    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-100-8152386bc65e>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [101]:

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
7/7 [==============================] - 1s 3ms/step - loss: 1.9528 - accuracy: 0.1644
Epoch 2/200
7/7 [==============================] - 0s 3ms/step - loss: 1.9440 - accuracy: 0.1736
Epoch 3/200
7/7 [==============================] - 0s 1ms/step - loss: 1.9129 - accuracy: 0.2124
Epoch 4/200
7/7 [==============================] - 0s 3ms/step - loss: 1.8132 - accuracy: 0.2235
Epoch 5/200
7/7 [==============================] - 0s 0s/step - loss: 1.7803 - accuracy: 0.4714
Epoch 6/200
7/7 [==============================] - 0s 0s/step - loss: 1.5802 - accuracy: 0.5135
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 1.6660 - accuracy: 0.4727
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 1.3972 - accuracy: 0.6291
Epoch 9/200
7/7 [==============================] - 0s 0s/step - loss: 1.3802 - accuracy: 0.5553
Epoch 10/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4158 - accuracy: 0.5486
Epoch 11/200
7/7 [==============

7/7 [==============================] - 0s 0s/step - loss: 0.0127 - accuracy: 1.0000
Epoch 86/200
7/7 [==============================] - 0s 0s/step - loss: 0.0502 - accuracy: 0.9929
Epoch 87/200
7/7 [==============================] - 0s 0s/step - loss: 0.1012 - accuracy: 0.9774
Epoch 88/200
7/7 [==============================] - 0s 0s/step - loss: 0.0432 - accuracy: 0.9929
Epoch 89/200
7/7 [==============================] - 0s 0s/step - loss: 0.0080 - accuracy: 1.0000
Epoch 90/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0888 - accuracy: 1.0000
Epoch 91/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0640 - accuracy: 1.0000
Epoch 92/200
7/7 [==============================] - 0s 0s/step - loss: 0.0840 - accuracy: 1.0000
Epoch 93/200
7/7 [==============================] - 0s 0s/step - loss: 0.0406 - accuracy: 1.0000
Epoch 94/200
7/7 [==============================] - 0s 0s/step - loss: 0.0817 - accuracy: 0.9837
Epoch 95/200
7/7 [=======================

7/7 [==============================] - 0s 0s/step - loss: 0.0394 - accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 0s/step - loss: 0.0267 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 0s/step - loss: 0.0090 - accuracy: 1.0000 
Epoch 171/200
7/7 [==============================] - 0s 0s/step - loss: 0.1387 - accuracy: 0.9316
Epoch 172/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0104 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 0s/step - loss: 0.0015 - accuracy: 1.0000 
Epoch 176/200
7/7 [==============================] - 0s 0s/step - loss: 0.0088 - accuracy: 1.0000
Epoch 177/200
7/7 [==============================] - 0s 0s/step - loss: 0.0299 - accuracy: 1.0000
Epoch 178/200
7/7 [==========

In [103]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('keywords.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [104]:
def clean_up_sentence(sentence):
    
    sentence_words = nltk.word_tokenize(sentence)
    
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [105]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [107]:
import tkinter
from tkinter import *
import matplotlib
matplotlib.use('Agg')


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="black", font=("Poppins", 14 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("BANKING INFORMATION")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


ChatLog = Text(base, bd=0, bg="white", height="8", width="100", font="Arial",)

ChatLog.config(state=DISABLED)


scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(base, font=("Poppins",12,'bold'), text="Send", width="16", height=6,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )


EntryBox = Text(base, bd=0, bg="white",width="40", height="6", font="Poppins")




scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()